## Discovered Words

In [2]:
%matplotlib inline 

import random
import re
from collections import Counter
import numpy as np

### Load Datasets

In [2]:
#targets

with open("/Users/carolineroper/Documents/School/Natural Language Processing/quasi-rnn-original/dev_target_seqs.txt", "r", encoding="utf8") as f:
    
    targets = []
    
    for i, line in enumerate(f):
        
        line = line.split('<JOIN>')
        targets.append(line[0])
        
#predictions        
    
with open("/Users/carolineroper/Documents/School/Natural Language Processing/quasi-rnn-original/decoded_original.txt", "r", encoding="utf8") as f:
    
    predictions = []
    s2=[]
    
    for i, line in enumerate(f):
        s1=(line.split('  '))
        #print(s1)
        for k, word in enumerate(s1):
            s2.append(" ".join(["".join(w.split(" ")) for w in word.split("  ")]))
        predictions.append(" ".join(s2))
        s2=[]
        
#training targets
        
with open("/Users/carolineroper/Documents/School/Natural Language Processing/quasi-rnn-original/train_target_seqs.txt", "r", encoding="utf8") as f:
    
    train_targets = []
    
    for i, line in enumerate(f):
        
        line = line.split('<JOIN>')
        train_targets.append(line[0])

#### Random Examples

In [18]:
randoms = random.sample(range(0,len(targets)), 30)

for i in randoms:
    print (targets[i], predictions[i])

#### Get Vocabulary of Training Targets

In [100]:
def normalizeString(s):
    s = s.lower().strip()
    s = re.sub(r"([.!?])", r" \1", s) #separates punctuation from the word
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s) #strips anything that isn't a character of punctuation
    return s

In [114]:
def find_vocabulary(seq, vocab_size):
    counter = Counter()
    for sentence in seq:
        counter.update(normalizeString(sentence).split())
    vocabulary = [count[0] for count in counter.most_common(vocab_size)]
    return vocabulary

In [115]:
train_vocab = find_vocabulary(train_targets, 200000)

In [117]:
len(train_vocab)

49545

In [105]:
result_vocab = find_vocabulary(predictions, 100000)

In [106]:
len(result_vocab)

3874

#### Get "Discovered" Words

In [118]:
oov = []

for word in result_vocab:
    if word not in train_vocab:
        oov.append(word)

In [155]:
def find_example(word):
    for i in range(len(predictions)):
        if word in normalizeString(predictions[i]).split():
            index_where_found = i
    if word in normalizeString(targets[index_where_found]).split():
        correctness = True
    else:
        correctness= False
    return word, targets[index_where_found], predictions[index_where_found], correctness

In [157]:
find_example('ethnosphere')

('ethnosphere',
 "And what we're doing is a series of journeys to the ethnosphere where we're going to take our audience to places of such cultural wonder that they cannot help but come away dazzled by what they have seen, and hopefully, therefore, embrace gradually, one by one, the central revelation of anthropology: that this world deserves to exist in a diverse way, that we can find a way to live in a truly multicultural, pluralistic world where all of the wisdom of all peoples can contribute to our collective well-being.\n",
 'And we take a series of rices in the ethnosphere, on the stage where we should be our audience to place that they were taking the public of space and hopefully they can take a food that the world is about the central office that the central offers that we can take a food in the wisdom of all people \n',
 True)

#### Find Correct "Discovered Words"

In [166]:
num_correct = 0
correct_discoveries = []
disc_targets = []
disc_predictions = []

for word in oov:
    word, target, prediction, correctness = find_example(word)
    if correctness == True:
        num_correct += 1
        correct_discoveries.append(word)
        disc_targets.append(target)
        disc_predictions.append(prediction)
        
num_correct

20

In [164]:
correct_discoveries

['ethnosphere',
 'burk',
 'monoamine',
 'bioland',
 'frosty',
 'barasana',
 'marta',
 'yanomami',
 'snavely',
 'pareto',
 'playoff',
 'katey',
 'degger',
 'hackteria',
 'profilers',
 'imploded',
 'ude',
 'lento',
 'buridan',
 'solon']

In [ ]:
#many of these are proper nouns or scientific terms, but a handful can be attributed to correctly
#compounding words, such as "profilers" "playoff" "ethnosphere" and "bioland", "imploded" perhaps

#### Find examples of Incorrect "Discovered Words"

In [172]:
incorrect_oov = list(set(oov).difference(set(correct_discoveries)))

In [195]:
thefile = open('incorrect_new_words.txt', 'w', encoding = 'utf-8')

randoms = random.sample(range(0,len(incorrect_oov)), 10)

for i in randoms:
    output = list(find_example(incorrect_oov[i]))
    #output = [str(x) for x in output]
    thefile.write("\n")
    for x in output[0:3]:
        thefile.write("\n%s" % x)